In [ ]:
import numpy as np
from collections import defaultdict


class CRF:
    def __init__(self, tags, feature_templates):
        """
        :param feature_templates: 特征模板列表
        :param tags: 标签列表
        """
        self.tags = tags
        self.feature_templates = feature_templates
        self.tag_to_idx = {tag: i for i, tag in enumerate(tags)}
        self.idx_to_tag = {i: tag for tag, i in self.tag_to_idx.items()}

        # 模型参数
        self.weights = defaultdict(float)  # 特征权重
        self.transition = np.random.randn(len(tags), len(tags)) * 0.01  # 转移矩阵

    @staticmethod
    def extract_features_helper(sentence, position, template):
        """提取特征模板中的偏移量"""
        context = []  # U08:%x[0,0]/%x[1,0]
        parts = template.split(":")[1].split("/")
        for part in parts:
            offset = int(part[3:-1].split(",")[0])
            idx = position + offset
            if idx < 0:
                context.append("<BEG>")
            elif idx >= len(sentence):
                context.append("<END>")
            else:
                context.append(sentence[idx])
        return context

    def extract_features(self, sentence, position, prev_tag, cur_tag):
        """根据特征模板提取特征"""
        features = []
        for template in self.feature_templates:
            # 处理Unigram特征
            if template.startswith("U"):
                context = self.extract_features_helper(sentence, position, template)
                features.append(f"{cur_tag}::{template}:{'/'.join(context)}")

            # 处理Bigram特征
            elif template.startswith("B"):
                if prev_tag is not None:
                    context = self.extract_features_helper(sentence, position, template)
                    features.append(f"{prev_tag}→{cur_tag}::{template}:{'/'.join(context)}")
        return features

    def forward_backward(self, sentence):
        """前向后向算法"""
        T = len(sentence)
        N = len(self.tags)

        # 初始化
        alpha = np.zeros((T, N))
        beta = np.zeros((T, N))

        # 前向算法
        # alpha_t(j)=\sum_{i=1}^N alpha_{t-1}(i) * T(i,j) * E_t(j)
        for t in range(T):
            for j in range(N):
                if t == 0:
                    features = self.extract_features(sentence, t, None, self.idx_to_tag[j])
                    alpha[t][j] = sum(self.weights[f] for f in features)
                else:
                    log_probs = []
                    for i in range(N):
                        trans = self.transition[i][j]  # T(i,j)
                        features = self.extract_features(sentence, t, self.idx_to_tag[i], self.idx_to_tag[j])
                        emit = sum(self.weights[f] for f in features)  # E_t(j)
                        log_probs.append(alpha[t - 1][i] + trans + emit)
                    alpha[t][j] = np.logaddexp.reduce(log_probs) if log_probs else -np.inf

        # 后向算法
        # beta_t(i)=\sum_{j=1}^N T(i,j) * E_{t+1}(j) * beta_{t+1}(j)
        for t in reversed(range(T)):
            for i in range(N):
                if t == T - 1:
                    beta[t][i] = 0
                else:
                    log_probs = []
                    for j in range(N):
                        trans = self.transition[i][j]  # T(i,j)
                        features = self.extract_features(sentence, t + 1, self.idx_to_tag[i], self.idx_to_tag[j])
                        emit = sum(self.weights[f] for f in features)  # E_{t+1}(j)
                        log_probs.append(trans + emit + beta[t + 1][j])
                    beta[t][i] = np.logaddexp.reduce(log_probs) if log_probs else -np.inf

        # 配分函数
        # Z=\sum_{j=1}^N alpha_T(j)
        log_Z = np.log(sum(np.exp(alpha[-1]))) if any(np.isfinite(alpha[-1])) else -np.inf
        return alpha, beta, log_Z

    def compute_gradient(self, sentence, true_tags):
        """计算梯度"""
        # 提取真实路径特征
        true_features = set()
        for t in range(len(sentence)):
            prev_tag = true_tags[t - 1] if t > 0 else None
            features = self.extract_features(sentence, t, prev_tag, true_tags[t])
            true_features.update(features)

        # 前向后向算法
        alpha, beta, log_Z = self.forward_backward(sentence)
        expected_features = defaultdict(float)

        # 计算特征期望
        # P(y_0=j|x)            = alpha_0(j) * E_0(j) / Z
        # P(y_{t-1}=i, y_t=j|x) = alpha_{t-1}(i) * T(i,j) * E_t(j) * beta_t(j) / Z
        for t in range(len(sentence)):
            for i in range(len(self.tags)):
                for j in range(len(self.tags)):
                    # 提取特征
                    cur_tag = self.idx_to_tag[j]
                    prev_tag = self.idx_to_tag[i] if t > 0 else None
                    features = self.extract_features(sentence, t, prev_tag, cur_tag)

                    # 计算概率
                    if t == 0:
                        prob = np.exp(alpha[t][j] + beta[t][j] - log_Z) if log_Z != -np.inf else 0
                    else:
                        trans_score = self.transition[i][j]
                        emit_score = sum(self.weights[f] for f in features)
                        prob = np.exp(alpha[t - 1][i] + trans_score + emit_score + beta[t][j] - log_Z) if log_Z != -np.inf else 0

                    # 累加特征期望
                    for f in features:
                        expected_features[f] += prob

        # 计算权重梯度
        # weight_grad[f] = true - expected
        weight_grad = defaultdict(float)
        for f in true_features:
            weight_grad[f] += 1
        for f in expected_features:
            weight_grad[f] -= expected_features[f]

        # 计算转移矩阵梯度
        # transition_grad[f] = true - expected
        transition_grad = np.zeros_like(self.transition)
        for t in range(1, len(sentence)):
            i = self.tag_to_idx[true_tags[t - 1]]
            j = self.tag_to_idx[true_tags[t]]
            transition_grad[i][j] += 1

            for i_model in range(len(self.tags)):
                for j_model in range(len(self.tags)):
                    features = self.extract_features(sentence, t, self.idx_to_tag[i_model], self.idx_to_tag[j_model])
                    prob = np.exp(alpha[t - 1][i_model] + self.transition[i_model][j_model] + sum(self.weights[f] for f in features) + beta[t][j_model] - log_Z) if log_Z != -np.inf else 0
                    transition_grad[i_model][j_model] -= prob

        return weight_grad, transition_grad, log_Z

    def train(self, sentences, true_tags_seq, max_iter=10, learning_rate=0.1):
        for iteration in range(max_iter):
            total_loss = 0
            for sentence, tags in zip(sentences, true_tags_seq):
                # 计算梯度
                weights_grad, transition_grad, log_Z = self.compute_gradient(sentence, tags)

                # 更新权重
                for f in weights_grad:
                    self.weights[f] += learning_rate * weights_grad[f]

                # 更新转移矩阵
                self.transition += learning_rate * transition_grad

                # 计算真实路径得分
                true_score = 0
                for t in range(len(sentence)):
                    prev_tag = tags[t - 1] if t > 0 else None
                    features = self.extract_features(sentence, t, prev_tag, tags[t])
                    true_score += sum(self.weights[f] for f in features)
                    if t > 0:
                        i = self.tag_to_idx[tags[t - 1]]
                        j = self.tag_to_idx[tags[t]]
                        true_score += self.transition[i][j]

                # 累加损失
                total_loss += log_Z - true_score

            print(f"Iteration {iteration}, Loss: {total_loss/len(sentences):.2f}")

    def viterbi_decode(self, sentence):
        """Viterbi算法解码"""
        T = len(sentence)
        N = len(self.tags)

        # 初始化
        viterbi = np.zeros((T, N))
        backpointers = np.zeros((T, N), dtype=int)

        # 初始步
        for j in range(N):
            features = self.extract_features(sentence, 0, None, self.idx_to_tag[j])
            viterbi[0][j] = sum(self.weights[f] for f in features)

        # 递推
        for t in range(1, T):
            for j in range(N):
                max_score = -np.inf
                best_tag = 0
                for i in range(N):
                    trans_score = self.transition[i][j]
                    features = self.extract_features(sentence, t, self.idx_to_tag[i], self.idx_to_tag[j])
                    emit_score = sum(self.weights[f] for f in features)
                    score = viterbi[t - 1][i] + trans_score + emit_score
                    if score > max_score:
                        max_score = score
                        best_tag = i
                viterbi[t][j] = max_score
                backpointers[t][j] = best_tag

        # 回溯
        best_path = [np.argmax(viterbi[-1])]
        for t in reversed(range(1, T)):
            best_path.append(backpointers[t][best_path[-1]])
        best_path.reverse()

        return [self.idx_to_tag[i] for i in best_path]


# 示例使用
# 定义特征模板
feature_templates = [
    "U00:%x[-2,0]",
    "U01:%x[-1,0]",
    "U02:%x[0,0]",
    "U03:%x[1,0]",
    "U04:%x[2,0]",
    "U05:%x[-2,0]/%x[-1,0]",
    "U06:%x[-1,0]/%x[0,0]",
    "U07:%x[-1,0]/%x[1,0]",
    "U08:%x[0,0]/%x[1,0]",
    "U09:%x[1,0]/%x[2,0]",
    "B00:%x[-2,0]",
    "B01:%x[-1,0]",
    "B02:%x[0,0]",
    "B03:%x[1,0]",
    "B04:%x[2,0]",
    "B05:%x[-2,0]/%x[-1,0]",
    "B06:%x[-1,0]/%x[0,0]",
    "B07:%x[-1,0]/%x[1,0]",
    "B08:%x[0,0]/%x[1,0]",
    "B09:%x[1,0]/%x[2,0]",
]

# 定义标签集
tags = ["O", "B-NAME", "I-NAME", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]

# 初始化CRF
crf = CRF(tags, feature_templates)

# 训练数据示例
train_sentences = [["张", "三", "在", "北京", "工作"], ["李", "四", "是", "腾讯", "员工"]]
train_tags = [["B-NAME", "I-NAME", "O", "B-LOC", "O"], ["B-NAME", "I-NAME", "O", "B-ORG", "O"]]

# 训练模型
crf.train(train_sentences, train_tags, max_iter=50)

# 预测新句子
test_sentence = ["王", "五", "来自", "上海"]
predicted_tags = crf.viterbi_decode(test_sentence)
print("预测结果:", list(zip(test_sentence, predicted_tags)))

Iteration 0, Loss: 1.99
Iteration 1, Loss: 0.18
Iteration 2, Loss: 1.28
Iteration 3, Loss: 1.10
Iteration 4, Loss: 2.11
Iteration 5, Loss: 1.01
Iteration 6, Loss: 2.15
Iteration 7, Loss: 1.73
Iteration 8, Loss: 1.48
Iteration 9, Loss: 2.28
Iteration 10, Loss: 1.11
Iteration 11, Loss: 2.35
Iteration 12, Loss: 1.72
Iteration 13, Loss: 1.61
Iteration 14, Loss: 2.29
Iteration 15, Loss: 1.14
Iteration 16, Loss: 2.43
Iteration 17, Loss: 1.66
Iteration 18, Loss: 1.71
Iteration 19, Loss: 2.27
Iteration 20, Loss: 1.18
Iteration 21, Loss: 2.47
Iteration 22, Loss: 1.59
Iteration 23, Loss: 1.80
Iteration 24, Loss: 2.23
Iteration 25, Loss: 1.23
Iteration 26, Loss: 2.49
Iteration 27, Loss: 1.51
Iteration 28, Loss: 1.90
Iteration 29, Loss: 2.18
Iteration 30, Loss: 1.28
Iteration 31, Loss: 2.49
Iteration 32, Loss: 1.43
Iteration 33, Loss: 1.99
Iteration 34, Loss: 2.13
Iteration 35, Loss: 1.34
Iteration 36, Loss: 2.47
Iteration 37, Loss: 1.35
Iteration 38, Loss: 2.09
Iteration 39, Loss: 2.07
Iteration 